In [2]:
#!/usr/bin/env python
from rosetta import *
init(extra_options='-include_sugars -override_rsd_type_limit -read_pdb_link_records -write_pdb_link_records')

initial_pose=pose_from_pdb('/Users/XT/Dropbox/Project/pilus_repacking.pdb')
#pose=pose_from_pdb('/Users/XT/Dropbox/Project/pilus_repacking.pdb')
pose=Pose()



# Pymol Mover
pmm=PyMOLMover()
#loop_domain=range(132-16,139+1-16)
scorefxn=get_score_function()
# MoveMap
mm=MoveMap()
mm.set_bb_true_range(132-16,139-16)   # inclusive
mm.set_branches(True)                 # sugar
mm.set_nu(True)                       # for Ring Mover

# Mover
smallmover=SmallMover(mm,1,1)
shearmover=ShearMover(mm,1,1)
minmover=MinMover()
minmover.movemap(mm)   
minmover.score_function(scorefxn)
minmover.min_type('dfpmin')
ringmover=RingConformationMover()
ringmover.movemap(mm)

# Job Distributor
jd=PyJobDistributor("0903decoys",1,scorefxn)


'''# log_acceptance_rate
f=open('log.txt','a')
f.write('count \t num_accepts \t acceptance_rate \n')
f.close()'''

"# log_acceptance_rate\nf=open('log.txt','a')\nf.write('count \t num_accepts \t acceptance_rate \n')\nf.close()"

In [4]:
# floppy tail algorithm
## 1. max angle for shear mover: 180 degree
count=0
#while not jd.job_complete:
for i in range(1):
    pose=Pose()
    pose.assign(initial_pose)
    #Monte Carlo
    mc=MonteCarlo(pose,scorefxn,1)         # change kT, half is ideal
    shearmover.angle_max('L',180)
    # task pack
    task_pack=standard_packer_task(pose)
    # 120 Cys skipped
    count+=1
    num_accepts=0
    '''for i in range(1):                   # min mover,50, 5%      ?set tolerance & type df    250
        for ii in range(19):               # shear+ring,19, 95%
            shearmover.apply(pose)
            #mc.boltzmann(pose)
            if mc.boltzmann(pose):
                num_accepts+=1
            ringmover.apply(pose)
            #mc.boltzmann(pose)
            if mc.boltzmann(pose):
                num_accepts+=1
            #pmm.apply(pose)
        minmover.apply(pose)       
        #pmm.apply(pose)
        #mc.boltzmann(pose)
        if mc.boltzmann(pose):
            num_accepts+=1'''

## 2. max angle for shear mover: 4 degree
    shearmover.angle_max('L',4)
    for i in range(1):                    # repacking    3%    90/8
        for ii in range(121,128+1-1):        # repacking    3%    90
            for iii in range(1):           # 2,2 min mover    3%    90
                for j in range(1):         # shear+ring+rotamer   95% 16/8
                    for k in range (121,128+1-1):  # turn on 121,128+1 one by one
                        shearmover.apply(pose)
                        ringmover.apply(pose)
                        #mc.boltzmann(pose)
                        if mc.boltzmann(pose):
                            num_accepts+=1
                        # reset task_pack
                        task_pack.restrict_to_repacking()
                        task_pack.temporarily_fix_everything()
                        for kk in range(116,119+1):
                            task_pack.temporarily_set_pack_residue(kk,True)   # turn on 116-119
                        task_pack.temporarily_set_pack_residue(k,True)
                        task_pack.temporarily_set_pack_residue(k+1,True)
                        # Rotamer Trials
                        rotamer_trials=RotamerTrialsMover(scorefxn,task_pack)
                        # print k, task_pack   --ok
                        rotamer_trials.apply(pose)         # should add more loops, *8
                        #dump pdb k
                        #mc.boltzmann(pose)
                        if mc.boltzmann(pose):
                            num_accepts+=1
                        #pmm.apply(pose)
                minmover.apply(pose)
                #pmm.apply(pose)
                #mc.boltzmann(pose)
                if mc.boltzmann(pose):
                    num_accepts+=1
            #repack
            task_pack.restrict_to_repacking()
            task_pack.temporarily_fix_everything()
            for kk in range(116,119+1):
                task_pack.temporarily_set_pack_residue(kk,True)
            task_pack.temporarily_set_pack_residue(ii,True)
            task_pack.temporarily_set_pack_residue(ii+1,True)
            pack_mover=PackRotamersMover(scorefxn,task_pack)
            pack_mover.apply(pose)                         # should add more loops, *8
            minmover.apply(pose)
            #mc.boltzmann(pose)
            if mc.boltzmann(pose):
                num_accepts+=1
            #pmm.apply(pose)

    #jd.output_decoy(pose)


    # Acceptance Rate
    print 'Total num of Monte Carlo trials is ', mc.total_trials()
    print 'Num_Accepts=', num_accepts
    acceptance_rate=float(num_accepts)/float(mc.total_trials())
    acceptance_rate=float('%.4f'%acceptance_rate)
    print 'Acceptance Rate for count',count,' is ', acceptance_rate
    f=open('log.txt','a')
    f.write(str(count)+'\t'+str( num_accepts)+' \t\t' +str(acceptance_rate)+' \n')
    f.close()



121 #Packer_Task

resid	pack?	design?	allowed_aas
1	FALSE	FALSE	ALA:NtermProteinFull
2	FALSE	FALSE	ALA
3	FALSE	FALSE	ILE
4	FALSE	FALSE	ALA
5	FALSE	FALSE	ILE
6	FALSE	FALSE	PRO
7	FALSE	FALSE	ALA
8	FALSE	FALSE	TYR
9	FALSE	FALSE	GLN
10	FALSE	FALSE	ASN
11	FALSE	FALSE	TYR
12	FALSE	FALSE	ILE
13	FALSE	FALSE	ALA
14	FALSE	FALSE	LYS
15	FALSE	FALSE	SER
16	FALSE	FALSE	GLN
17	FALSE	FALSE	VAL
18	FALSE	FALSE	SER
19	FALSE	FALSE	THR
20	FALSE	FALSE	GLY
21	FALSE	FALSE	LEU
22	FALSE	FALSE	ALA
23	FALSE	FALSE	ASP
24	FALSE	FALSE	ILE
25	FALSE	FALSE	THR
26	FALSE	FALSE	ALA
27	FALSE	FALSE	GLY
28	FALSE	FALSE	LYS
29	FALSE	FALSE	THR
30	FALSE	FALSE	ASN
31	FALSE	FALSE	ALA
32	FALSE	FALSE	GLU
33	FALSE	FALSE	THR
34	FALSE	FALSE	LYS
35	FALSE	FALSE	LEU
36	FALSE	FALSE	ALA
37	FALSE	FALSE	GLU
38	FALSE	FALSE	GLY
39	FALSE	FALSE	LEU
40	FALSE	FALSE	THR
41	FALSE	FALSE	ALA
42	FALSE	FALSE	ALA
43	FALSE	FALSE	LEU
44	FALSE	FALSE	THR
45	FALSE	FALSE	ASP
46	FALSE	FALSE	VAL
47	FALSE	FALSE	GLU
48	FALSE	FALSE	ALA
49	FALSE	FALSE	LEU
50	FALSE	FA

In [7]:
pose=Pose()
pose.assign(initial_pose)
    #Monte Carlo
mc=MonteCarlo(pose,scorefxn,1)         # change kT, half is ideal
    # task pack
task_pack=standard_packer_task(pose)
for k in range (121,128+1-1):  # turn on 121,128+1 one by one
    task_pack.restrict_to_repacking()
    task_pack.temporarily_fix_everything()
    for kk in range(116,119+1):
        task_pack.temporarily_set_pack_residue(kk,True)   # turn on 116-119
    task_pack.temporarily_set_pack_residue(k,True)
    task_pack.temporarily_set_pack_residue(k+1,True)
                        # Rotamer Trials
    print k,'rotamer begin'
    rotamer_trials.apply(pose)
    print k,'rotamer done'
    pose.dump_pdb('/Users/XT/Dropbox/Project/Structure/test_%d.pdb'%(k))

121 rotamer begin
121 rotamer done
122 rotamer begin
122 rotamer done
123 rotamer begin
123 rotamer done
124 rotamer begin
124 rotamer done
125 rotamer begin
125 rotamer done
126 rotamer begin
126 rotamer done
127 rotamer begin
127 rotamer done
